In [1]:
import numpy as np
import sys
import os
import neuro
import risp
import random

from encode import Chromagram
from encode import Encoder
from data_loader import data_loader
import risp
import neuro
import eons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score


1.) Innitialize the neuroprocessor

In [2]:
risp_config = {
  "leak_mode": "all",
  "min_weight": -1,
  "max_weight": 1,
  "min_threshold": -1,
  "max_threshold": 1,
  "max_delay": 5,
  "discrete": False
}

proc = risp.Processor(risp_config) # RISP processor

net = neuro.Network() # Neuro network
net.set_properties(proc.get_network_properties()) # Set network properties


2.) Load the data

In [3]:
# First, split the data into training+validation and testing sets
X_temp, X_test, y_temp, y_test = train_test_split(data_loader().chroma_files, data_loader().numerical_label_roots, test_size=0.2, random_state=42)

# Now, split the training+validation set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# One-hot encode the labels
OneHot = OneHotEncoder(sparse_output=False)
y_train_encoded = OneHot.fit_transform(np.array(y_train).reshape(-1, 1))
y_val_encoded = OneHot.transform(np.array(y_val).reshape(-1, 1))
y_test_encoded = OneHot.transform(np.array(y_test).reshape(-1, 1))

3.) create template for eons

In [4]:
n_inputs = 12
n_hidden = 360
n_outputs = 12
n_neurons = n_inputs + n_hidden + n_outputs
n_synapses = 1000


moa = neuro.MOA()
moa.seed(42)

In [5]:
def create_neuron(neuron_id, net, moa):
    neuron = net.add_node(neuron_id)
    net.randomize_node_properties(moa, neuron)
    return neuron

In [6]:
for i in range(n_inputs):
    neuron = create_neuron(i, net, moa)
    neuron.set("Threshold",0.75)
    net.add_input(i)
    
for i in range(n_outputs):
    neuron = create_neuron(i+n_inputs, net, moa)
    neuron.set("Threshold",0.75)
    net.add_output(i)
    
for i in range(n_hidden):
    neuron = create_neuron(i+n_inputs+n_outputs, net, moa)

In [7]:
print("Neuron 0's input ID:", net.get_node(0).input_id)
print("Neuron 1's input ID:", net.get_node(1).input_id)

Neuron 0's input ID: 0
Neuron 1's input ID: 1


In [8]:
for i in range(n_synapses):
    source = random.randint(0,n_neurons-1)
    dest = random.randint(0,n_neurons-1)
    synapse = net.add_or_get_edge(source, dest)
    net.randomize_edge_properties(moa, synapse)

4.) set up the eons

In [9]:
eo_params = {
    "population_size": 100,
    "starting_nodes": n_neurons,
    "starting_edges": n_neurons,
    "multi_edges": 0.05,
    "merge_rate": .01,
    "crossover_rate": 0.5,
    "mutation_rate": 0.9,
    "add_node_rate": 0.5,
    "add_edge_rate": 0.9,
    "delete_node_rate": 0.4,
    "delete_edge_rate": 0.8,
    "node_params_rate": 1.5,
    "edge_params_rate": 1.5,
    "net_params_rate": .01,
    "num_mutations": 5,
    "random_factor": 0.05,
    "num_best": 3,

    "selection_type": "tournament",
    "tournament_size_factor": 0.1,
    "tournament_best_net_factor": 0.9,
    "node_mutations": { "Threshold": 1.0 },
    "net_mutations": { },
    "edge_mutations": { "Weight": 0.5, "Delay": 0.5 },
}


In [10]:
evolver = eons.EONS(eo_params)
evolver.set_template_network(net)

pop = evolver.generate_population(eo_params,42)

In [11]:
def get_prediction(proc, x):
    # Load the chroma data from the npy file
    encoder=Encoder(x)
    
    proc.clear_activity()
    proc.apply_spikes(encoder.spikes)
    proc.run(encoder.time_steps * encoder.num_bins) # you might adjust this duration based on your needs
    
    # Decoding the output to get the predicted label. You might need to adjust this
    predicted_index = proc.output_count_max(n_outputs)[0]
    # Convert index to one-hot encoded format
    one_hot_prediction = np.zeros(n_outputs)
    one_hot_prediction[predicted_index] = 1.0
    
    return one_hot_prediction 


In [12]:
def fitness(proc, net, X, y):
    proc.load_network(net)
    
    # Set up output tracking
    for i in range(n_outputs):
        proc.track_neuron_events(i)
    
    y_predict = [get_prediction(proc, x) for x in X]
    return accuracy_score(y_predict, y)


In [13]:
max_generations = 100
vals = []

for generation in range(max_generations):
    # Evaluate fitness of all networks in the population
    fitnesses = [fitness(proc, net.network, X_train, y_train_encoded) for net in pop.networks]
    
    # Track and print best fitness in the current generation
    best_fitness = max(fitnesses)
    vals.append(best_fitness)
    print(f"Generation {generation + 1} Best Fitness: {best_fitness:.4f}")
    
    # Produce the next generation based on the current population's fitness
    pop = evolver.do_epoch(pop, fitnesses, eo_params)

# Optionally, you can evaluate and print the performance of the best network on the test set here.

print("Evolution finished!")